# EDA — Poêles à pellets & météoObjectif : construire les hypothèses et les vérifications nécessaires pour relier météo, cycles d'allumage et consommation de granulés.

## Plan express1. Charger et inspecter la structure des données (météo + cycles utilisateur)2. Formuler des hypothèses (ex. température ↔ consommation)3. Préparer les features temporelles et météo4. Visualiser les distributions clés5. Tester les relations (corrélations, comparaisons de moyennes)6. Synthétiser les enseignements opérationnels

In [ ]:
# Imports principauximport pandas as pdimport numpy as npimport seaborn as snsimport matplotlib.pyplot as pltfrom pathlib import Pathfrom scipy import statssns.set_theme(style="whitegrid")pd.set_option("display.max_columns", 50)

In [ ]:
# Paramètres à adapter en fonction de la localisation du fichier sourceRAW_DATA_PATH = Path("data/raw/pellet_cycles.csv")# Colonnes attendues (ajuster selon le jeu de données réel)TIME_COL = "timestamp"CONS_COL = "pellet_kg"TEMP_COL = "temperature_ext"HUM_COL = "humidite"WIND_COL = "vent_kmh"PRICE_COL = "prix_euro"TARGET_TEMP_COL = "temperature_int_cible"

In [ ]:
# Chargement des données# Adapter les paramètres parse_dates / sep si besoindf = pd.read_csv(    RAW_DATA_PATH,    parse_dates=[TIME_COL],    infer_datetime_format=True,)# Normalisation des noms de colonnes : espaces -> underscoredf.columns = [c.strip().lower().replace(" ", "_") for c in df.columns]df.head()

## 1) Structure & qualité des donnéesVérifier la complétude, les types, et la granularité temporelle.

In [ ]:
# Aperçu structurelinfo_df = df.info()display(info_df)# Statistiques descriptives brutesdisplay(df.describe(datetime_is_numeric=True).T)# Taux de valeurs manquantesmissing_rate = df.isna().mean().sort_values(ascending=False)display(missing_rate.to_frame("missing_ratio"))

## 2) Hypothèses clés à vérifier- H1 : plus la température extérieure est basse, plus la consommation de pellets augmente.- H2 : le vent et l'humidité amplifient les pics de consommation (pertes thermiques + confort ressenti).- H3 : les cycles du matin sont plus longs/coûteux que ceux du soir à température extérieure équivalente.- H4 : la surconsommation apparaît quand la consigne intérieure est trop élevée par rapport à l'inertie du logement.

In [ ]:
# Préparation des features temporellesdf["date"] = df[TIME_COL].dt.datedf["jour_semaine"] = df[TIME_COL].dt.day_name()df["heure"] = df[TIME_COL].dt.hourdf["mois"] = df[TIME_COL].dt.monthdf["saison"] = pd.cut(    df["mois"],    bins=[0, 3, 6, 9, 12],    labels=["Hiver", "Printemps", "Été", "Automne"],    include_lowest=True,)# Agrégations rapides par jourdaily = df.groupby("date").agg(    conso_totale=(CONS_COL, "sum"),    temperature_moy=(TEMP_COL, "mean"),    humidite_moy=(HUM_COL, "mean"),    vent_moy=(WIND_COL, "mean"),    prix_moy=(PRICE_COL, "mean") if PRICE_COL in df.columns else (CONS_COL, "size"),).reset_index()daily.head()

## 3) Distributions de baseObjectif : détecter asymétries, valeurs aberrantes et niveaux usuels.

In [ ]:
fig, axes = plt.subplots(2, 2, figsize=(12, 10))sns.histplot(df[CONS_COL], kde=True, ax=axes[0, 0], color="#6A4C93")axes[0, 0].set_title("Distribution consommation (kg)")sns.histplot(df[TEMP_COL], kde=True, ax=axes[0, 1], color="#1982C4")axes[0, 1].set_title("Distribution température extérieure")sns.boxplot(x="saison", y=CONS_COL, data=df, ax=axes[1, 0], palette="muted")axes[1, 0].set_title("Conso par saison")sns.boxplot(x="jour_semaine", y=CONS_COL, data=df, ax=axes[1, 1], palette="coolwarm")axes[1, 1].set_title("Conso par jour de la semaine")plt.xticks(rotation=45)plt.tight_layout()plt.show()

## 4) Relation météo ↔ consommation (H1 + H2)- Visualisation : nuage de points + régression locale- Mesure : corrélation de Spearman (robuste aux non-linéarités)

In [ ]:
fig, axes = plt.subplots(1, 3, figsize=(18, 5))sns.regplot(x=TEMP_COL, y=CONS_COL, data=df, lowess=True, ax=axes[0], scatter_kws={"alpha": 0.3})axes[0].set_title("Température vs consommation")sns.regplot(x=WIND_COL, y=CONS_COL, data=df, lowess=True, ax=axes[1], scatter_kws={"alpha": 0.3}, color="#FF595E")axes[1].set_title("Vent vs consommation")sns.regplot(x=HUM_COL, y=CONS_COL, data=df, lowess=True, ax=axes[2], scatter_kws={"alpha": 0.3}, color="#8AC926")axes[2].set_title("Humidité vs consommation")plt.tight_layout()plt.show()# Corrélations Spearmanmeteo_cols = [TEMP_COL, WIND_COL, HUM_COL]corr_spearman = df[[CONS_COL] + meteo_cols].corr(method="spearman")corr_spearman[CONS_COL]

## 5) Effet horaire & cycles (H3)Comparer les profils matin (6h-10h) vs soir (18h-22h) à température extérieure comparable.

In [ ]:
df["plage_horaire"] = pd.cut(    df["heure"], bins=[-1, 5, 10, 17, 22, 24], labels=["Nuit", "Matin", "Midi", "Soir", "Tard"], include_lowest=True)fig, ax = plt.subplots(figsize=(10, 5))sns.boxplot(x="plage_horaire", y=CONS_COL, data=df, ax=ax, palette="pastel")ax.set_title("Consommation par plage horaire")plt.show()# Test de différence de moyennes (matin vs soir)matin = df[df["plage_horaire"] == "Matin"][CONS_COL]soir = df[df["plage_horaire"] == "Soir"][CONS_COL]t_stat, p_val = stats.ttest_ind(matin.dropna(), soir.dropna(), equal_var=False)print(f"Test t Matin vs Soir — statistique={t_stat:.2f}, p-value={p_val:.3g}")

## 6) Surchauffe potentielle (H4)Approche : comparer la consommation aux écarts entre température intérieure cible et température extérieure.

In [ ]:
if TARGET_TEMP_COL in df.columns:    df["delta_t"] = df[TARGET_TEMP_COL] - df[TEMP_COL]    fig, ax = plt.subplots(figsize=(8, 5))    sns.regplot(x="delta_t", y=CONS_COL, data=df, lowess=True, ax=ax, scatter_kws={"alpha": 0.3})    ax.set_title("Écart consigne - extérieur vs consommation")    plt.show()    corr_delta = df[["delta_t", CONS_COL]].corr(method="spearman").iloc[0, 1]    print(f"Corrélation Spearman delta_T ↔ conso : {corr_delta:.3f}")else:    print("Colonne de consigne intérieure absente : renseigner TARGET_TEMP_COL pour tester H4.")

## 7) Coût & efficacitéConversion de la consommation en € et recherche de surconsommation évitable.

In [ ]:
if PRICE_COL in df.columns:    df["cout_cycle"] = df[CONS_COL] * df[PRICE_COL]    daily_costs = df.groupby("date")["cout_cycle"].sum()    print(f"Coût journalier moyen : {daily_costs.mean():.2f} €")else:    print("Prix par kg non fourni : ajouter la colonne PRICE_COL pour monétiser les économies.")

## 8) Synthèse interprétative (à compléter après exécution)- Facteurs majeurs observés : …- Plages horaires critiques : …- Potentiel d'économie (kg / €) : …- KPI candidats : consommation prédite vs réelle, surconsommation évitée, coût évité.